In [2]:
import torch
from torchvision import transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import optim

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [5]:
file_0 = np.load("corruptmnist/train_0.npz", allow_pickle= True)

In [6]:
file_0['images'].shape

(5000, 28, 28)

In [7]:
train_paths = [f"corruptmnist/train_{i}.npz" for i in range(5)]

In [8]:
X_train = np.concatenate([np.load(train_file)["images"] for train_file in train_paths])
Y_train = np.concatenate([np.load(train_file)["labels"] for train_file in train_paths])

X_test = np.load("corruptmnist/test.npz")["images"]
Y_test = np.load("corruptmnist/test.npz")["labels"]

In [9]:
X_train.shape

(25000, 28, 28)

In [10]:
X_test.shape

(5000, 28, 28)

In [12]:
class MNISTdata(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x.float(), y
    
    def __len__(self):
        return len(self.data)

In [13]:
train = MNISTdata(X_train, Y_train, transform=transform)
trainloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

test = MNISTdata(X_test, Y_test, transform=transform)
testloader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

In [14]:
next(iter(trainloader))[0].shape

torch.Size([64, 1, 28, 28])

In [23]:
import torch
import torch.nn.functional as F
from torch import nn

class MyAwesomeModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        ''' Builds a feedforward network with arbitrary hidden layers.
        
            Arguments
            ---------
            input_size: integer, size of the input layer
            output_size: integer, size of the output layer
            hidden_layers: list of integers, the sizes of the hidden layers
        
        '''
        super().__init__()
        # Input to a hidden layer
        self.num_classes = 10
        
        # Your code here!
        self.arch = nn.Sequential(
            nn.Conv2d(in_channels=1,
                      out_channels=64, 
                      kernel_size=3, 
                      padding=1,
                      stride=1),
            # output dim (64, 28, 28)
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, 
                         stride=2),
            # output dim (64, 14, 14)
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, 
                      out_channels=128, 
                      kernel_size=5, 
                      padding=2),
            nn.Dropout2d(p=0.1),
            # output dim (128, 14, 14)
            nn.MaxPool2d(kernel_size=2,
                         stride=2),
            # output dim (128, 7, 7)
            nn.ReLU(inplace=True)
        )
        
        # fully connected output layers
        # [(W−K+2P)/S]+1
        self.fc1_features = 128*7*7
        self.fc1 = nn.Linear(in_features=self.fc1_features, 
                             out_features=500)
        self.fc2 = nn.Linear(in_features=500,
                             out_features=10)
        


    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        x = self.arch(x)
        x = x.view(-1, self.fc1_features)
        #x = self.dropout(relu(self.l_1(x)))
        x = F.relu(self.fc1(x))
        
        return F.log_softmax(self.fc2(x), dim=1)


In [56]:
next(iter(trainloader))[1].shape

torch.Size([64])

In [24]:
model = MyAwesomeModel(784, 10, [512, 256, 128])
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
def validation(model, testloader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in testloader:

        # images = images.resize_(images.size()[0], 784)

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ## Calculating the accuracy 
        # Model's output is log-softmax, take exponential to get the probabilities
        ps = torch.exp(output)
        # Class with highest probability is our predicted class, compare with true label
        equality = (labels.data == ps.max(1)[1])
        # Accuracy is number of correct predictions divided by all predictions, just take the mean
        accuracy += equality.type_as(torch.FloatTensor()).mean()

    return test_loss, accuracy

def train(model, trainloader, testloader, criterion, optimizer=None, epochs=5, print_every=40):
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    steps = 0
    running_loss = 0
    for e in range(epochs):
        # Model in training mode, dropout is on
        model.train()
        for images, labels in trainloader:
            steps += 1
            # Flatten images into a 784 long vector
            # images.resize_(images.size()[0], 784)
            
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

            if steps % print_every == 0:
                # Model in inference mode, dropout is off
                model.eval()
                
                # Turn off gradients for validation, will speed up inference
                with torch.no_grad():
                    test_loss, accuracy = validation(model, testloader, criterion)
                
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
                
                running_loss = 0
                
                # Make sure dropout and grads are on for training
                model.train()

In [28]:
train(model, trainloader, testloader, criterion, optimizer, epochs=2)

Epoch: 1/2..  Training Loss: 0.219..  Test Loss: 0.304..  Test Accuracy: 0.910
Epoch: 1/2..  Training Loss: 0.192..  Test Loss: 0.294..  Test Accuracy: 0.905
Epoch: 1/2..  Training Loss: 0.142..  Test Loss: 0.264..  Test Accuracy: 0.914
Epoch: 1/2..  Training Loss: 0.133..  Test Loss: 0.247..  Test Accuracy: 0.920
Epoch: 1/2..  Training Loss: 0.129..  Test Loss: 0.273..  Test Accuracy: 0.910
Epoch: 1/2..  Training Loss: 0.143..  Test Loss: 0.217..  Test Accuracy: 0.930
Epoch: 1/2..  Training Loss: 0.110..  Test Loss: 0.179..  Test Accuracy: 0.936
Epoch: 1/2..  Training Loss: 0.113..  Test Loss: 0.190..  Test Accuracy: 0.937
Epoch: 1/2..  Training Loss: 0.109..  Test Loss: 0.185..  Test Accuracy: 0.938
Epoch: 2/2..  Training Loss: 0.107..  Test Loss: 0.182..  Test Accuracy: 0.943
Epoch: 2/2..  Training Loss: 0.073..  Test Loss: 0.181..  Test Accuracy: 0.944
Epoch: 2/2..  Training Loss: 0.051..  Test Loss: 0.155..  Test Accuracy: 0.953
Epoch: 2/2..  Training Loss: 0.068..  Test Loss: 0.1

In [37]:
for x,y in iter(trainloader):
    print(y)

tensor([5, 6, 7, 9, 1, 1, 1, 1, 6, 5, 5, 4, 3, 6, 3, 4, 6, 2, 6, 8, 0, 6, 4, 5,
        4, 0, 3, 7, 7, 6, 2, 5, 2, 3, 0, 0, 0, 2, 6, 8, 2, 3, 4, 1, 3, 0, 6, 5,
        7, 1, 6, 1, 2, 5, 8, 4, 9, 0, 1, 9, 7, 6, 5, 4])
tensor([2, 0, 5, 5, 0, 4, 0, 0, 1, 9, 2, 2, 5, 1, 1, 0, 9, 6, 0, 5, 9, 1, 9, 7,
        2, 9, 3, 1, 8, 1, 8, 3, 5, 5, 3, 6, 6, 8, 6, 3, 4, 6, 1, 1, 7, 1, 4, 5,
        8, 5, 9, 1, 6, 4, 1, 7, 3, 2, 3, 6, 6, 8, 6, 0])
tensor([6, 2, 7, 2, 5, 9, 7, 2, 8, 4, 2, 6, 5, 9, 9, 4, 8, 6, 1, 0, 6, 8, 0, 6,
        0, 9, 3, 9, 8, 0, 7, 5, 8, 6, 3, 7, 5, 0, 1, 7, 2, 3, 4, 8, 8, 5, 1, 9,
        9, 0, 8, 2, 8, 2, 1, 4, 1, 9, 7, 5, 8, 1, 2, 2])
tensor([8, 4, 9, 4, 5, 0, 7, 2, 6, 0, 4, 1, 3, 5, 7, 9, 0, 7, 9, 4, 6, 2, 9, 6,
        1, 8, 1, 2, 7, 3, 9, 1, 1, 0, 1, 5, 4, 5, 3, 1, 0, 0, 0, 4, 2, 5, 7, 4,
        4, 2, 9, 8, 3, 2, 6, 8, 6, 5, 8, 2, 8, 0, 7, 7])
tensor([4, 2, 5, 9, 8, 3, 9, 6, 3, 4, 3, 6, 9, 4, 6, 9, 1, 0, 9, 4, 9, 4, 4, 7,
        2, 6, 2, 3, 1, 1, 8, 3, 0, 8, 4, 6, 4, 0, 1,